In [14]:
# 필요한 라이브러리 다시 로드
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import json

# JSON 파일 다시 로드
file_path = "webtoon_naver_crawling.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 데이터프레임 생성
df = pd.DataFrame(data)

# first_episode 날짜 변환
df['first_episode'] = pd.to_datetime(df['first_episode'], format='%Y.%m.%d', errors='coerce')

# 오늘 날짜 기준 일수 계산
today = datetime.today()
df['days_since_release'] = (today - df['first_episode']).dt.days

# 숫자형 변환
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["likes"] = pd.to_numeric(df["likes"], errors="coerce")
df["episode"] = pd.to_numeric(df["episode"], errors="coerce")
df['days_since_release'] = pd.to_numeric(df["days_since_release"], errors="coerce")

# Box-Cox 변환 (음수 방지)
df["rating_boxcox"], lambda_rating = stats.boxcox(df["rating"] - df["rating"].min() + 1)

# 이상치 처리 (Q3 기준)
q3_li = df["likes"].quantile(0.95)
df["likes_qt"] = df["likes"].apply(lambda x: q3_li if x > q3_li else x)

q3_ep = df["episode"].quantile(0.95)
df["episode_qt"] = df["episode"].apply(lambda x: q3_ep if x > q3_ep else x)

# Min-Max Scaling 함수
def min_max_scaling(series):
    min_val = series.min()
    max_val = series.max()
    if pd.isna(min_val) or pd.isna(max_val) or min_val == max_val:
        return series
    return (series - min_val) / (max_val - min_val)

# 정규화 적용
df["rating_norm"] = min_max_scaling(df["rating_boxcox"])
df["likes_log"] = np.log1p(df["likes_qt"])
df["likes_norm"] = min_max_scaling(df["likes_log"])
df["episode_norm"] = min_max_scaling(df["episode_qt"])
df["days_since_release_norm"] = min_max_scaling(df["days_since_release"])

# 장르별 가중치 설정
genre_weights = {
    "드라마": {"rating": 0.4, "likes": 0.4, "episode": 0.1, "days_since_release": -0.1},
    "로맨스": {"rating": 0.5, "likes": 0.3, "episode": 0.1, "days_since_release": -0.1},
    "판타지": {"rating": 0.35, "likes": 0.4, "episode": 0.1, "days_since_release": -0.15},
    "스릴러": {"rating": 0.4, "likes": 0.4, "episode": 0.1, "days_since_release": -0.1},
    "액션": {"rating": 0.35, "likes": 0.45, "episode": 0.1, "days_since_release": -0.05},
    "개그": {"rating": 0.35, "likes": 0.35, "episode": 0.1, "days_since_release": -0.2},
    "일상": {"rating": 0.4, "likes": 0.3, "episode": 0.1, "days_since_release": -0.2},
    "무협/사극": {"rating": 0.4, "likes": 0.4, "episode": 0.1, "days_since_release": -0.1},
    "로판":{"rating": 0.5, "likes": 0.3, "episode": 0.1, "days_since_release": -0.1},
    "스포츠":{"rating": 0.4, "likes": 0.4, "episode": 0.1, "days_since_release": -0.1},
    "감성":{"rating": 0.4, "likes": 0.4, "episode": 0.1, "days_since_release": -0.1}
}

# 장르별 가중치 적용한 점수 계산
def calculate_score(row):
    genre = row["genre"]
    weights = genre_weights.get(genre, {"rating": 0.4, "likes": 0.4, "episode": 0.1, "days_since_release": -0.1})
    
    score = (
        row["rating_norm"] * weights["rating"] +
        row["likes_norm"] * weights["likes"] +
        row["episode_norm"] * weights["episode"] +
        row["days_since_release_norm"] * weights["days_since_release"]
    )
    return score

df["first_score"] = df.apply(calculate_score, axis=1)

# 장르별 min-max scaling 적용
df["score"] = df.groupby("genre")["first_score"].transform(lambda x: min_max_scaling(x))

# 최종 정렬 (score 기준)
df = df.sort_values(by="score", ascending=False)

# 필요없는 열 정리
df['first_episode'] = df['first_episode'].dt.strftime('%Y.%m.%d')
df = df.drop(columns=["rating_boxcox", "rating_norm", "likes_log", "likes_norm", "episode_norm", "days_since_release_norm", "days_since_release","likes_qt","episode_qt", "first_score"])

# 결과 출력
import ace_tools_open as tools
tools.display_dataframe_to_user(name="장르별 min-max scaling 적용 웹툰 순위", dataframe=df)


장르별 min-max scaling 적용 웹툰 순위


In [15]:
# 특정 특수문자 (\n, \r, \') 제거 함수 정의
def clean_text(text):
    if isinstance(text, str):
        return text.replace("\n", "").replace("\r", "").replace("\\", "").strip()
    return text

# 모든 문자열 열에 대해 특수문자 제거 적용
df = df.map(clean_text)

# JSON 파일로 저장
with open("webtoon_naver_all.json", "w", encoding="utf-8") as f:
    json.dump(df.to_dict(orient="records"), f, indent=4, ensure_ascii=False)

In [16]:
import re

# 특정 특수문자 (\n, \r, \') 제거 함수 정의
def clean_text(text):
    if isinstance(text, str):
        return text.replace("\n", "").replace("\r", "").replace("\\", "").strip()
    return text

# 모든 문자열 열에 대해 특수문자 제거 적용
df = df.map(clean_text)

# 장르별로 데이터를 JSON 형식으로 다시 저장
output_files_json_cleaned = {}

for genre, group in df.groupby("genre"):
    safe_genre = re.sub(r'[^\w]', '_', genre)  # 특수문자 제거 및 안전한 파일명 변환
    filename = f"webtoon_naver_{safe_genre}.json"
    group.to_json(filename, orient="records", force_ascii=False, indent=4)
    output_files_json_cleaned[genre] = filename

# 생성된 JSON 파일 목록 출력
output_files_json_cleaned


{'감성': 'webtoon_naver_감성.json',
 '개그': 'webtoon_naver_개그.json',
 '드라마': 'webtoon_naver_드라마.json',
 '로맨스': 'webtoon_naver_로맨스.json',
 '로판': 'webtoon_naver_로판.json',
 '무협/사극': 'webtoon_naver_무협_사극.json',
 '스릴러': 'webtoon_naver_스릴러.json',
 '스포츠': 'webtoon_naver_스포츠.json',
 '액션': 'webtoon_naver_액션.json',
 '일상': 'webtoon_naver_일상.json',
 '판타지': 'webtoon_naver_판타지.json'}